# Download data collected and stored on our Google Drive

In [1]:
from google_drive_downloader import GoogleDriveDownloader

In [2]:
def download_gd_data_from_dict(dictionary: dict):
    for i,j in dictionary.items():
        GoogleDriveDownloader.download_file_from_google_drive(file_id=j,
                                            dest_path=f"../datasets/raw_data/{i}/{i}.zip",
                                            unzip=True,
                                            showsize=True,
                                            overwrite=False)

## Source: GUIGK

In [3]:
gugik = {'voi':'1BZCmADIZhJuf1_Jh-f6D8vSpI8p5-2wd',
        'pov':'1wX99dmNUbiEKYKh-qAfxDipT9oC6DLzE',
        'com':'1URjb9NM6Fm_qES5kC4QPPXZGERzarUIa'}

download_gd_data_from_dict(gugik)

## Source: OSM

In [4]:
osm = {'mazowieckie':'195E_n9JlgavFWp4mbaOCHAYKFWziBkc0',
        'pomorskie':'146rCLx7-9RupOXXywm60Ug43LCP1SJiT',
        'malopolskie':'1KG6uPhCZ-jKDgEpBU46WKHXVG_Mc-dBS'}

download_gd_data_from_dict(osm)

339.0 MiB                                                                                                                                                                                                                                                                                                                     Done.
Unzipping...Done.
125.8 MiB                                                                                                                                      Done.
Unzipping...Done.
224.6 MiB                                                                                                                                                                                                                          Done.
Unzipping...Done.


# Scrape pickup points data from websites

## Source: Bliskapaczka.pl